In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
csv = 'clean_tweet.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,text,account_type
0,we have sitting democrat us senator on trial f...,Right
1,marshawn lynch arrives to game in anti trump s...,Right
2,daughter of fallen navy sailor delivers powerf...,Right
3,just in president trump dedicates presidents c...,Right
4,respecting our national anthem standforouranthem,Right


In [3]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202315 entries, 0 to 202314
Data columns (total 2 columns):
text            202315 non-null object
account_type    202315 non-null object
dtypes: object(2)
memory usage: 3.1+ MB


In [4]:
x = my_df.text
y = my_df.account_type

In [5]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
from tqdm import tqdm
#tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
import multiprocessing
from sklearn import utils

In [8]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result

In [9]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

/Users/computer/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [10]:
len(all_x_w2v)

202315

#### Distributed Bag of Words

In [21]:
model_ug_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, workers=1, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

/Users/computer/miniconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 202315/202315 [00:00<00:00, 2060377.55it/s]


In [22]:
%%time
for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha

100%|██████████| 202315/202315 [00:00<00:00, 2408173.81it/s]


CPU times: user 4min 7s, sys: 2.04 s, total: 4min 9s
Wall time: 4min 16s


In [ ]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

In [ ]:
train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation, 100)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow, y_train)

In [ ]:
clf.score(validation_vecs_dbow, y_validation)

In [ ]:
model_ug_dbow.save('d2v_model_ug_dbow.doc2vec')
model_ug_dbow = Doc2Vec.load('d2v_model_ug_dbow.doc2vec')

In [ ]:
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

#### Distributed memory - concatenated

In [ ]:
cores = multiprocessing.cpu_count()
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in range(30):
    model_ug_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha

In [ ]:
#model_ug_dmc = Doc2Vec.load('d2v_model_ug_dmc.doc2vec')

In [ ]:
model_ug_dmc.most_similar('good')


In [ ]:
model_ug_dmc.most_similar('happy')

In [ ]:
model_ug_dmc.most_similar('facebook')

In [ ]:
model_ug_dmc.most_similar('trump')

In [ ]:
model_ug_dmc.most_similar(positive=['bigger', 'small'], negative=['big'])

In [ ]:
train_vecs_dmc = get_vectors(model_ug_dmc, x_train, 100)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_validation, 100)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dmc, y_train)

In [ ]:
clf.score(validation_vecs_dmc, y_validation)

In [ ]:
model_ug_dmc.save('d2v_model_ug_dmc.doc2vec')
model_ug_dmc = Doc2Vec.load('d2v_model_ug_dmc.doc2vec')
model_ug_dmc.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

#### Distributed memory (mean)

#### Word2Vec

In [19]:
import gensim
#from gensim.models import Word2Vec
model = gensim.models.Word2Vec (my_df, size=15000, window=10, min_count=2, workers=10)
model.train(my_df,total_examples=len(my_df),epochs=10)

(6, 160)

In [20]:
w1 = "Right"
model.wv.most_similar (positive=w1)

KeyError: "word 'Right' not in vocabulary"